In [4]:
import pandas as pd

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [48]:
def format_time(x):
    if not any(i in x for i in ['DNF', 'DNS']):
        if ':' in x:
            return round(float(str(x).split(':')[1]) + (60 * float(str(x).split(':')[0])), 3) if x != 0 else 0
        else:
            return(round(float(x), 3))
    else:
        return x

In [47]:
races = pd.read_csv(path+'data/races.csv')
qualifying = pd.read_csv(path+'data/qualifying.csv')
constructors = pd.read_csv(path+'data/constructor_standings.csv')
drivers = pd.read_csv(path+'data/driver_standings.csv')
results = pd.read_csv(path+'data/results.csv')
starting_grid = pd.read_csv(path+'data/starting-grid.csv')

### Redundant until new weather collection method is implemented
# weather = pd.read_csv(path+'data/weather.csv')

In [49]:
results = results[results.no.notna()]

starting_grid = starting_grid.drop(['driver', 'constructor'], axis=1)

In [23]:
laps1 = pd.read_csv(path+'data/laps-2014-2017.csv')
laps2 = pd.read_csv(path+'data/laps-2017-22.csv')

In [5]:
laps = pd.concat([laps1, laps2]).drop_duplicates()

laps.shape

(168937, 6)

In [6]:
laps[laps.duplicated()]

,season,round,lap,driver,position,time


In [7]:
laps['time'] = laps.time.apply(lambda x: format_time(str(x)))

In [8]:
# laps.to_csv(path+'data/laps.csv', index=False)

In [50]:
def lookup (df, team, points):
    df['lookup1'] = df.season.astype(str) + df[team] + df['round'].astype(str)
    df['lookup2'] = df.season.astype(str) + df[team] + (df['round']-1).astype(str)
    new_df = df.merge(df[['lookup1', points]], how = 'left', left_on='lookup2',right_on='lookup1')
    new_df.drop(['lookup1_x', 'lookup2', 'lookup1_y'], axis = 1, inplace = True)
    new_df.rename(columns = {points+'_x': points+'_after_race', points+'_y': points}, inplace = True)
    new_df[points].fillna(0, inplace = True)
    return new_df

In [51]:
for col in ['driver_points', 'driver_wins', 'driver_standings_pos']:
    drivers = lookup(drivers, 'driver', col)

# drivers.drop(['driver_points_after_race', 'driver_wins_after_race', 'driver_standings_pos_after_race'], axis=1, inplace=True)

In [52]:
for col in ['constructor_points', 'constructor_wins', 'constructor_standings_pos']:
    constructors = lookup(constructors, 'constructor', col)

# constructors.drop(['constructor_points_after_race', 'constructor_wins_after_race', 'constructor_standings_pos_after_race'], axis=1, inplace=True)

In [11]:
mean_time = laps.groupby(['season', 'round', 'driver']).agg({'time': 'mean'}).reset_index()
mean_time['time'] = mean_time.time.apply(lambda x: round(x, 3))

NameError: name 'laps' is not defined

In [56]:
df1 = pd.merge(races, results, how='inner', on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time', 'grid'], axis=1)
df2 = pd.merge(df1, drivers, how='left', on=['season', 'round', 'driver']) 
df3 = pd.merge(df2, constructors, how='left', on=['season', 'round', 'constructor'])
df4 = pd.merge(df3, starting_grid, how='inner', on=['season', 'round', 'no'])
df4.rename(columns = {'grid': 'starting_grid'}, inplace=True)

merged_df = pd.merge(df4, qualifying, how='inner', on=['season', 'round', 'circuit_id', 'driver'])
merged_df.rename(columns = {'fastest_time': 'qual_time', 'grid': 'qualifying_pos'}, inplace=True)

merged_df.shape

(3589, 31)

In [57]:
merged_df

,season,round,circuit_id,country,lat,long,date,driver,no,date_of_birth,...,constructor_wins_after_race,constructor_standings_pos_after_race,constructor_points,constructor_wins,constructor_standings_pos,starting_grid,qualifying_pos,qual_time,stage,q_delta
0,2014,1,albert_park,Australia,-37.8497,144.9680,2014-03-16 06:00:00,rosberg,6.0,1985-06-27,...,1,2,0.0,0.0,0.0,3,3,104.595,q3,0.000
1,2014,1,albert_park,Australia,-37.8497,144.9680,2014-03-16 06:00:00,kevin_magnussen,20.0,1992-10-05,...,0,1,0.0,0.0,0.0,4,4,105.745,q3,-6.088
2,2014,1,albert_park,Australia,-37.8497,144.9680,2014-03-16 06:00:00,button,22.0,1980-01-19,...,0,1,0.0,0.0,0.0,10,11,104.437,q2,0.206
3,2014,1,albert_park,Australia,-37.8497,144.9680,2014-03-16 06:00:00,alonso,14.0,1981-07-29,...,0,3,0.0,0.0,0.0,5,5,105.819,q3,-6.014
4,2014,1,albert_park,Australia,-37.8497,144.9680,2014-03-16 06:00:00,bottas,77.0,1989-08-28,...,0,4,0.0,0.0,0.0,15,10,108.147,q3,3.916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3584,2022,21,interlagos,Brazil,-23.7036,-46.6997,2022-11-13 18:00:00,latifi,6.0,1995-06-29,...,0,10,8.0,0.0,10.0,18,16,75.095,q1,3.421
3585,2022,21,interlagos,Brazil,-23.7036,-46.6997,2022-11-13 18:00:00,tsunoda,22.0,2000-05-11,...,0,9,35.0,0.0,9.0,20,19,76.264,q1,4.590
3586,2022,21,interlagos,Brazil,-23.7036,-46.6997,2022-11-13 18:00:00,norris,4.0,1999-11-13,...,0,5,146.0,0.0,5.0,6,4,72.263,q3,0.589
3587,2022,21,interlagos,Brazil,-23.7036,-46.6997,2022-11-13 18:00:00,kevin_magnussen,20.0,1992-10-05,...,0,8,36.0,0.0,8.0,8,1,71.674,q3,0.000


In [59]:
merged_df['fastest_lap'] = merged_df['fastest_lap'].apply(lambda x: format_time(str(x)))

In [60]:
merged_df[['fastest_lap', 'qual_time']].head()

,fastest_lap,qual_time
0,92.478,104.595
1,93.066,105.745
2,92.917,104.437
3,93.186,105.819
4,92.616,108.147


In [61]:
merged_df[['season','round', 'driver', 'starting_grid', 'qualifying_pos','podium', 'fastest_lap']].query('season == 2015 & round == 1').sort_values('starting_grid')

,season,round,driver,starting_grid,qualifying_pos,podium,fastest_lap
378,2015,1,hamilton,1,1,1,90.945
379,2015,1,rosberg,2,2,2,91.092
381,2015,1,massa,3,3,4,91.719
380,2015,1,vettel,4,4,3,91.457
389,2015,1,raikkonen,5,5,12,91.426
383,2015,1,ricciardo,6,7,6,92.797
386,2015,1,sainz,7,8,9,92.872
391,2015,1,grosjean,8,9,14,NaN
392,2015,1,maldonado,9,10,15,NaN
382,2015,1,nasr,10,11,5,92.612


In [62]:
merged_df.columns

Index(['season', 'round', 'circuit_id', 'country', 'lat', 'long', 'date',
       'driver', 'no', 'date_of_birth', 'nationality', 'constructor', 'podium',
       'fastest_lap', 'driver_points_after_race', 'driver_wins_after_race',
       'driver_standings_pos_after_race', 'driver_points', 'driver_wins',
       'driver_standings_pos', 'constructor_points_after_race',
       'constructor_wins_after_race', 'constructor_standings_pos_after_race',
       'constructor_points', 'constructor_wins', 'constructor_standings_pos',
       'starting_grid', 'qualifying_pos', 'qual_time', 'stage', 'q_delta'],
      dtype='object')

In [63]:
null_cols = merged_df.columns[merged_df.isna().any()].tolist()
# merged_df.query()
null_rows = merged_df[merged_df.isna().any(axis=1)]
null_rows = null_rows[['season', 'round', 'driver', 'podium', 'qual_time','fastest_lap', 'stage']]

In [64]:
null_rows

,season,round,driver,podium,qual_time,fastest_lap,stage
14,2014,1,maldonado,16,NaN,94.766,NaN
18,2014,1,kobayashi,21,105.867,NaN,q2
19,2014,1,ricciardo,22,104.548,NaN,q3
40,2014,2,perez,22,122.511,NaN,q2
74,2014,4,maldonado,14,NaN,103.067,NaN
...,...,...,...,...,...,...,...
3530,2022,18,sainz,19,89.361,NaN,q3
3531,2022,18,albon,20,91.311,NaN,q1
3550,2022,19,sainz,20,94.356,NaN,q3
3587,2022,21,kevin_magnussen,19,71.674,NaN,q3


In [66]:
# fill/drop nulls
### Data is being lossed here

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins', 'constructor_standings_pos']:
    merged_df[col].fillna(0, inplace=True)
    merged_df[col] = merged_df[col].map(lambda x: int(x))
    
merged_df.dropna(inplace=True)

merged_df.shape

(3394, 31)

In [20]:
merged_df.to_csv(path+'data/merged.csv', index=False)